In [22]:
import requests
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm_notebook as tqdm
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
import selenium
import pickle
import time
from datetime import datetime
import unicodedata
import random
import urllib.parse
from selenium.webdriver.common.keys import Keys

In [24]:
from peewee import *
from playhouse.sqliteq import SqliteQueueDatabase

db = SqliteQueueDatabase('db.sqlite3',
    use_gevent=False,  # Use the standard library "threading" module.
    autostart=False,  # The worker thread now must be started manually.
    queue_max_size=64)


class Items(Model):
    name = TextField()
    photo = TextField()
    url = TextField()
    cost = IntegerField()
    
    message_url = TextField(default="")
    response = BooleanField(default = True)
    surveillance = BooleanField(default = False)
    
    
    class Meta:
        database = db
        
class Tasks(Model):
    name = TextField()
    url = TextField()
    max_cost = IntegerField()
    message_text = TextField()

    class Meta:
        database = db


db.start()
db.connect()
# db.create_tables([Items,Tasks])
# db.stop() #  Wait for create_tables() to complete

In [3]:
options =  Options()
    # options.set_headless(True)
driver = webdriver.Firefox(options=options)
ips=[]

In [4]:
driver.get("https://www.avito.ru/sankt-peterburg?q=iphone")

In [5]:
page  = driver.page_source 
soup = BeautifulSoup(page, 'html5lib')

In [6]:
def send_mail(url):
    driver.get(url)
    time.sleep(2)
    page  = driver.page_source 
    soup = BeautifulSoup(page, 'html5lib')
    
    clas = soup.find("button",{"data-marker":"messenger-button/button"})
    if clas is None:
        clas = soup.find("a",{"data-marker":"messenger-button/link"})["class"]
    else:
        clas=clas["class"]
    clas=" ".join(clas)
    
    script = f'document.getElementsByClassName("{clas}")[0].click()'
    driver.execute_script(script)
    time.sleep(4)
    url = driver.current_url
    driver.find_element_by_tag_name("textarea").send_keys("Добрый вечер")# Keys.ENTER
    time.sleep(1)
    return url

In [7]:
send_mail("https://www.avito.ru/sankt-peterburg/telefony/iphone_8_x_xr_7_6s_6_4_5s_se_7_plus_8_plus_6s_plus_1867170159")

NoSuchElementException: Message: Unable to locate element: textarea


In [ ]:
page  = driver.page_source 
soup = BeautifulSoup(page, 'html5lib')

In [ ]:
len(soup.find_all("div",{"data-marker":"message"}))

In [ ]:
def bash(bash_сommand):
    import subprocess
    process = subprocess.Popen(bash_сommand.split(), stdout=subprocess.PIPE)
    return process.communicate()

def alarm(text):
    requests.get("https://alarmerbot.ru/?key=df548f-61ac83-624ea4&message=Hello%20human%21")

items_urls = []
def get_items(url):
    driver.get(url)
    page  = driver.page_source 
    soup = BeautifulSoup(page, 'html5lib')
    
    for item in soup.find_all("div",{"class":"item__line"}):
        url = "https://www.avito.ru"+item.a["href"]
        cost = item.find("div",{"class":"snippet-price-row"}).span.text.replace("\n","").replace(" ","")
        cost = re.findall(r'\d+', cost)
        img= item.img["src"]
        if len(cost)>0:
            cost=cost[0]
        else:
            cost=0
        cost = int(cost)
        name = item.h3.text
        
        if Items.select().where(Items.url==url).count() == 0:
            Items.create(name=name, cost=cost, url=url,photo=img)
            items_urls.append(url)
            print("New",name ,img)
            
            try:
                send_mail(url)
            except Exception as e:
                print(e)
            break

In [ ]:
get_items("https://www.avito.ru/sankt-peterburg?q=iphone")

In [ ]:
t = datetime.now()
for n in range(150):
    driver.refresh()
    print((datetime.now()-t).seconds)

In [ ]:
(datetime.now()-t).seconds